In [ ]:
!python3 -m pip install --upgrade pip
!python3 -m pip install konlpy  

     |████████████████████████████████| 1.6 MB 7.9 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 19.4 MB 238 kB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
     |████████████████████████████████| 448 kB 93.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
!apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.14).
git is already the newest version (1:2.17.1-1ubuntu0.8).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Installing automake (A dependency for mecab-ko)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InR

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from konlpy.tag import Mecab
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/dataset/dacon/technic_classifier/'

train=pd.read_csv(file_path+'train.csv')
test=pd.read_csv(file_path+'test.csv')
sample_submission=pd.read_csv(file_path+'sample_submission.csv')

train.fillna('NoContent', inplace=True)
test.fillna('NoContent', inplace=True)

In [ ]:
# train=pd.read_csv('./data/train.csv')
# test=pd.read_csv('./data/test.csv')

# train.fillna('NoContent', inplace=True)
# test.fillna('NoContent', inplace=True)

In [ ]:
# 유사한 내역사업명 통합 

def united_contents(text):
  text = re.sub('\(R&D\)','',text)
  text = re.sub('\(\w+\)','',text)
  text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z ]','',text)
  return text

train['united_내역사업명'] = train['내역사업명'].apply(united_contents)
test['united_내역사업명'] = test['내역사업명'].apply(united_contents)

In [ ]:
# 토크나이저 

mecab = Mecab()

# mecab의 품사 분류 중 학습데이터에 사용할 품사 리스트
target_pos = ['NNG','NNP','NNB','NR','MM','IC',
              'SY','SL','VV+ETN','VV+ETN','MM','JKV',
              'XR','XPN','VCP+ETM','VV+EC','VV+EC',
              #예외허용
              'XSN','JX']

def fixed_mecab_tokenizer(text):

  # 텍스트 정제

  text = re.sub('가\.',' ',str(text))
  text = re.sub('나\.',' ',text)
  text = re.sub('\d*\.',' ',text)
  text = re.sub('\<br\>',' ',text)
  text = re.sub('[\*\n]',' ',text)
  text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\d\- ]"," ", text)
  text = text.lower()
  text = re.sub(' +',' ',text)
  text = text.strip()
  if len(text) == 0 :
    text = 'nocontent'
  
  text = re.sub(' ',' 뼝 ',text) # 기존 띄어쓰기를 활용하기 위해 다른 단어로 대체
                                 # 대체 하더라도 mecab에 영향이 적은 명사 사용
  # mecab tokenizer(띄어쓰기 단위로 분리)

  text = mecab.pos(text)         
  text = re.sub('\[','',str(text))
  text = re.sub('\]','',text)
  text = text.split(', (\'뼝\', \'NNG\'), ')

  space_split = []
  for i in range(len(text)):
    space_token = []
    tokens = text[i][1:-1].split('), (')
    for j in range(len(tokens)):
      pair = re.sub('\'','',tokens[j])
      space_token.append(pair.split(', '))


    space_split.append(space_token)

  target_tokens = ''
 
 # 학습에 사용할 토큰만 추출

  for i in range(len(space_split)):
    space_tokens = space_split[i]
    tokens = []
    
    if (len(space_tokens) == 0) or ((len(space_tokens) == 1) and (space_tokens[0][1] not in ['NNG','NNP','NNB','SL'])):
      continue
    elif ((len(space_tokens) == 1) and (space_tokens[0][0] == '수')):
      continue

    target_token = []

    for i in range(len(space_tokens)):

      if (space_tokens[i][0] == '뼝'):
        space_tokens[i][1] = '제외'
        continue
      
      elif space_tokens[i][1] in target_pos:
        if (i != 0) and (space_tokens[i][0] == '표') and (space_tokens[i-1][0] == '최종목'):
          space_tokens[i][1] = 'MM'
          
        elif (i != 0) and (space_tokens[i][1] == 'SN') and (space_tokens[i-1][1] in ['SY','SL']):
          space_tokens[i][1] = 'MM'
          
        elif (i != 0) and (space_tokens[i][1] == 'XSN'):         
          if (space_tokens[i][0] in ['적','성','치']):
            space_tokens[i][1] = 'MM'
          else:
            continue

        elif (i != 0) and (space_tokens[i][1] == 'JX'):     
          if (space_tokens[i][0] in []):
            space_tokens[i][1] = 'MM'
          else:
            continue
                
        target_token.append(space_tokens[i])

    united_token=''

    if len(target_token) != 0:  
      for j in range(len(target_token)):
        united_token += target_token[j][0]

        if j != len(target_token)-1:
          if (target_token[j][1] in ['NNG','NNB','NNP']) and (target_token[j+1][1] in ['NNG','NNP','NNB']) and (len(target_token[j][0]) != 1) and (len(target_token[j+1][0]) != 1):
            united_token += ' '

    target_tokens += united_token
    target_tokens += ' '

  target_tokens = re.sub(' +',' ',target_tokens)
  target_tokens = target_tokens.strip()

  return target_tokens.split()

In [ ]:
# 사용 컬럼 토큰화. 내역사업명, 과제명, 요약문_한글키워드, 요약문_연구목표 4개의 컬럼 사용. 

train['token_내역사업명'] = train['united_내역사업명'].apply(fixed_mecab_tokenizer)
train['token_과제명'] = train['과제명'].apply(fixed_mecab_tokenizer)
train['token_요약문_한글키워드'] = train['요약문_한글키워드'].apply(fixed_mecab_tokenizer)
train['token_요약문_연구목표'] = train['요약문_연구목표'].apply(fixed_mecab_tokenizer)

test['token_내역사업명'] = test['united_내역사업명'].apply(fixed_mecab_tokenizer)
test['token_과제명'] = test['과제명'].apply(fixed_mecab_tokenizer)
test['token_요약문_한글키워드'] = test['요약문_한글키워드'].apply(fixed_mecab_tokenizer)
test['token_요약문_연구목표'] = test['요약문_연구목표'].apply(fixed_mecab_tokenizer)

In [ ]:
# stop word

del_words1 = ['주요','목표','개선','향상','과제','개발','기술','연구','선행','시스템','다음','매우','이용','로서','기존',
              '극복','장점','단점','문제','문제점','각각','관련','부분','간의','이상','달성','주된내용','내용','국내',
              '해외','최종','사용','효과','기관','제공','이하','목적','년차','필요','때문','로부터','최종적','만개','크게',
              '모두','의미','여러','하나로','중심','한번','위해','차년','년도','연도','최종목표']

del_words2 = ['안팎','개념','역할','중심적','또한','개월','취급','변화',
            '고려','가량','일반적','지원','종합적','도출','방안','검토',
            '강화','기업','바탕','현장','조건','방법','성숙','해당과정','위해',
            '검토','확립','차년','진행','과정','구축','연구개발','결과물',
            '검증','방안','분석','발굴','규명','적용','실제','거나','연구원',
            '운영','확보','대하','수','능력','평가','성과','발표','도록','마련',
            '따라서','대해','제시','통합적','종합적','말함','살피','최근','대두','일반',
            '발전','연구개발','경우','주어','기초','년간','담당','낼','대처','성의','핵심',
            '주목표','상위','하위','기대','시기','증대','증가','감소','고자','이바지',
            '이전','현재', '이번']

del_words = list(set(del_words1 + del_words2))

def add_filter(text):
  return [x for x in text if x not in del_words]


train['filter_내역사업명'] = train['token_내역사업명'].apply(add_filter)
train['filter_과제명'] = train['token_과제명'].apply(add_filter)
train['filter_요약문_한글키워드'] = train['token_요약문_한글키워드'].apply(add_filter)
train['filter_요약문_연구목표'] = train['token_요약문_연구목표'].apply(add_filter)

test['filter_내역사업명'] = test['token_내역사업명'].apply(add_filter)
test['filter_과제명'] = test['token_과제명'].apply(add_filter)
test['filter_요약문_한글키워드'] = test['token_요약문_한글키워드'].apply(add_filter)
test['filter_요약문_연구목표'] = test['token_요약문_연구목표'].apply(add_filter)

In [ ]:
# 결과물 제출 파일은 3개의 모델의 결과값을 보팅으로 앙상블하여 사용
# 사용 컬럼은 모두 같음.

# 모델 1 : 1개의 인풋을 받는 1D CNN 모델. 이 모델은 요약문_연구목표를 제외한 3개의 컬럼 사용.

# 모델 2 : 2개의 인풋을 받는 1D CNN 모델. 인풋 1:내역사업명, 과제명, 요약문_한글키워드 / 인풋 2: 요약문_연구목표 

# 모델 3 : 1개의 인풋을 받는 1D CNN 모델. 4개의 컬럼을 사용, 연구목표의 길이가 너무 길기 때문에 길이는 좀 짧게 만듬

In [ ]:
# 모델1 1인풋 1D CNN

train['model1_input'] =  train['filter_내역사업명'] + train['filter_과제명']  + train['filter_요약문_한글키워드']
test['model1_input'] =  test['filter_내역사업명'] + test['filter_과제명']  + test['filter_요약문_한글키워드']

In [ ]:
# 기본 토큰화 
tokenizer=Tokenizer(oov_token = 'OOV')

tokenizer.fit_on_texts(list(train['model1_input']))
word_vocab = tokenizer.word_index

vocab_size = len(word_vocab)+2

In [ ]:
# 단어 사전 크기 확인 

threshold = 3
total_cnt = len(tokenizer.word_index) 
rare_cnt = 0 
total_freq = 0 
rare_freq = 0 

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

print('예상 단어집합 크기 :',total_cnt - rare_cnt)
words_num = total_cnt - rare_cnt

단어 집합(vocabulary)의 크기 : 99316
등장 빈도가 2번 이하인 희귀 단어의 수: 49929
단어 집합에서 희귀 단어의 비율: 50.27286640621853
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.9737215323965311
예상 단어집합 크기 : 49387


In [ ]:
# 단어 사전 크기 설정

tokenizer=Tokenizer(num_words=words_num,oov_token = 'OOV')

tokenizer.fit_on_texts(list(train['model1_input']))
word_vocab = tokenizer.word_index

vocab_size = len(word_vocab)+2

In [ ]:
# 0이 아닌 데이터 2번 복사하여 추가, 추가로 오답률이 높은 label 2번 복사하여 추가 
temp = train.copy()
temp1 = temp[temp['label'] != 0]
temp2 = temp[(temp['label'] == 11)|(temp['label'] == 12)|(temp['label'] == 13)|(temp['label'] == 17)|(temp['label'] == 20)|(temp['label'] == 23)|(temp['label'] == 25)|(temp['label'] == 26)|(temp['label'] == 37)|(temp['label'] == 38)]

temp = pd.concat([temp, temp1], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp1], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp2], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp2], axis = 0).reset_index(drop = True)

In [ ]:
train_length = train['model1_input'].apply(len)
train_length.head()

# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))

number = 98
print('인풋 길이 {}%: {}'.format(number,np.percentile(train_length, number)))

인풋 길이 제 1 사분위: 16.0
인풋 길이 제 3 사분위: 24.0
인풋 길이 98%: 33.0


In [ ]:
# 벡터라이징

max_len = 33

model1_train_encoded = tokenizer.texts_to_sequences(list(temp['model1_input']))
model1_train_input = pad_sequences(model1_train_encoded, maxlen=max_len, padding='post')

model1_test_encoded = tokenizer.texts_to_sequences(list(test['model1_input']))
model1_test_input = pad_sequences(model1_test_encoded, maxlen=max_len, padding='post')

# 레이블 설정 
labels=np.array(temp['label'])

In [ ]:
# 불균형 데이터에 대한 대한으로 focal loss를 사용
# 마지막 레이어에서 softmax를 사용하므로 logit이 아닌 확률 값을 바로 사용

def focal_loss_with_out_logit(labels, y_pred, gamma=2):
    labels = tf.squeeze(labels)  
    labels = tf.cast(tf.one_hot(labels, depth=y_pred.shape[1]), dtype=tf.float32)

    loss = -labels * ((1 - y_pred) ** gamma) * tf.math.log(y_pred)
    loss = tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return loss

In [ ]:
embedding_dim = 512

model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len, mask_zero=True),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv1D(embedding_dim, kernel_size=5, padding='valid', activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
 
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.GlobalMaxPooling1D(),

    tf.keras.layers.Dense(128, activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(46, activation='softmax')
])

# 모델 컴파일
model1.compile(loss=focal_loss_with_out_logit,         
              optimizer= tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 33, 512)           50850816  
_________________________________________________________________
dropout (Dropout)            (None, 33, 512)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 29, 512)           1311232   
_________________________________________________________________
batch_normalization (BatchNo (None, 29, 512)           2048      
_________________________________________________________________
activation (Activation)      (None, 29, 512)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 512)           0         
_________________________________________________________________
global_max_pooling1d (Global (None, 512)               0

In [ ]:
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 20)
mc = ModelCheckpoint('best_model1.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

num_epochs = 250
history = model1.fit(model1_train_input, labels, epochs=num_epochs, batch_size=1024, verbose=1, validation_split=0.2, callbacks=[es, mc])

Epoch 1/250
195/195 [==============================] - 29s 67ms/step - loss: 0.6356 - accuracy: 0.8147 - val_loss: 6.6189 - val_accuracy: 0.0000e+00

Epoch 00001: val_accuracy improved from -inf to 0.00000, saving model to best_model1.h5
Epoch 2/250
195/195 [==============================] - 13s 65ms/step - loss: 0.1075 - accuracy: 0.9359 - val_loss: 5.1813 - val_accuracy: 0.0000e+00

Epoch 00002: val_accuracy did not improve from 0.00000
Epoch 3/250
195/195 [==============================] - 13s 65ms/step - loss: 0.0558 - accuracy: 0.9615 - val_loss: 0.8586 - val_accuracy: 0.2697

Epoch 00003: val_accuracy improved from 0.00000 to 0.26969, saving model to best_model1.h5
Epoch 4/250
195/195 [==============================] - 13s 65ms/step - loss: 0.0393 - accuracy: 0.9720 - val_loss: 0.0749 - val_accuracy: 0.9476

Epoch 00004: val_accuracy improved from 0.26969 to 0.94763, saving model to best_model1.h5
Epoch 5/250
195/195 [==============================] - 13s 65ms/step - loss: 0.0314

In [ ]:
model1.load_weights('best_model1.h5')

model1_pred=model1.predict(model1_test_input)
model1_pred=tf.argmax(model1_pred, axis=1)
sample_submission['model1_pred']=model1_pred

In [ ]:
# 모델 2 인풋 설정 

train['model2_vocab'] = train['filter_내역사업명'] + train['filter_과제명'] + train['filter_요약문_한글키워드'] + train['filter_요약문_연구목표']

tokenizer=Tokenizer(oov_token = 'OOV')

tokenizer.fit_on_texts(list(train['model2_vocab']))
word_vocab = tokenizer.word_index

vocab_size = len(word_vocab)+2

In [ ]:
threshold = 10
total_cnt = len(tokenizer.word_index) 
rare_cnt = 0 
total_freq = 0
rare_freq = 0 

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value
    
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

print('예상 단어집합 크기 :',total_cnt - rare_cnt)
words_num = total_cnt - rare_cnt

단어 집합(vocabulary)의 크기 : 223271
등장 빈도가 9번 이하인 희귀 단어의 수: 175688
단어 집합에서 희귀 단어의 비율: 78.68823089429438
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 3.5352407826532484
예상 단어집합 크기 : 47583


In [ ]:
tokenizer=Tokenizer(num_words=words_num,oov_token = 'OOV')

tokenizer.fit_on_texts(list(train['model2_vocab']))
word_vocab = tokenizer.word_index

vocab_size = len(word_vocab)+2

In [ ]:
train['model2_input_1'] = train['filter_내역사업명'] + train['filter_과제명'] + train['filter_요약문_한글키워드']
train['model2_input_2'] = train['filter_요약문_연구목표']

In [ ]:
test['model2_input_1'] = test['filter_내역사업명'] + test['filter_과제명'] + test['filter_요약문_한글키워드']
test['model2_input_2'] = test['filter_요약문_연구목표']

In [ ]:
train_length = train['model2_input_1'].apply(len)

print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))

number = 98
print('인풋 길이 {}%: {}'.format(number,np.percentile(train_length, number)))

인풋 길이 제 1 사분위: 16.0
인풋 길이 제 3 사분위: 24.0
인풋 길이 98%: 33.0


In [ ]:
train_length = train['model2_input_2'].apply(len)

print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))

number = 98
print('인풋 길이 {}%: {}'.format(number,np.percentile(train_length, number)))

인풋 길이 제 1 사분위: 24.0
인풋 길이 제 3 사분위: 72.0
인풋 길이 98%: 178.0


In [ ]:
# 0이 아닌 데이터 2번 복사하여 추가, 추가로 오답률이 높은 label 2번 복사하여 추가 
temp = train.copy()
temp1 = temp[temp['label'] != 0]
temp2 = temp[(temp['label'] == 11)|(temp['label'] == 12)|(temp['label'] == 13)|(temp['label'] == 17)|(temp['label'] == 20)|(temp['label'] == 23)|(temp['label'] == 25)|(temp['label'] == 26)|(temp['label'] == 37)|(temp['label'] == 38)]

temp = pd.concat([temp, temp1], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp1], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp2], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp2], axis = 0).reset_index(drop = True)

In [ ]:
max_len = [33, 175]

#트레인 인풋 
model2_train_encoded_1 = tokenizer.texts_to_sequences(list(temp['model2_input_1']))
model2_train_input_1 = pad_sequences(model2_train_encoded_1, maxlen=max_len[0], padding='post')

model2_train_encoded_2 = tokenizer.texts_to_sequences(list(temp['model2_input_2']))
model2_train_input_2 = pad_sequences(model2_train_encoded_2, maxlen=max_len[1], padding='post')

model2_train_input_1 = tf.convert_to_tensor(model2_train_input_1, dtype=tf.float32)
model2_train_input_2 = tf.convert_to_tensor(model2_train_input_2, dtype=tf.float32)

model2_train_input = []
model2_train_input.append(model2_train_input_1)
model2_train_input.append(model2_train_input_2)

# 테스트 인풋
model2_test_encoded_1 = tokenizer.texts_to_sequences(list(test['model2_input_1']))
model2_test_input_1 = pad_sequences(model2_test_encoded_1, maxlen=max_len[0], padding='post')

model2_test_encoded_2 = tokenizer.texts_to_sequences(list(test['model2_input_2']))
model2_test_input_2 = pad_sequences(model2_test_encoded_2, maxlen=max_len[1], padding='post')

model2_test_input_1 = tf.convert_to_tensor(model2_test_input_1, dtype=tf.float32)
model2_test_input_2 = tf.convert_to_tensor(model2_test_input_2, dtype=tf.float32)

model2_test_input = []
model2_test_input.append(model2_test_input_1)
model2_test_input.append(model2_test_input_2)


# 레이블 
labels=np.array(temp['label'])

In [ ]:
# 모델2 설정

embedding_dim = [512, 512]

def convolutional_model(embedding_dim = embedding_dim, max_len = max_len, num_labels = 46):
    
    train_input_0 = tf.keras.Input(shape=(max_len[0],))
    train_input_1 = tf.keras.Input(shape=(max_len[1],))


    x0 = tf.keras.layers.Embedding(vocab_size, embedding_dim[0], input_length=max_len[0], mask_zero=True)(train_input_0)
    x0 = tf.keras.layers.Dropout(0.2)(x0)

    x0 = tf.keras.layers.Conv1D(embedding_dim[0], kernel_size=3, padding='valid', activation=None)(x0)
    x0 = tf.keras.layers.BatchNormalization()(x0)
    x0 = tf.keras.layers.Activation('relu')(x0)
    x0 = tf.keras.layers.Dropout(0.2)(x0)
    x0 = tf.keras.layers.GlobalMaxPooling1D()(x0)

    x0 = tf.keras.layers.Dense(128, activation=None)(x0)
    x0 = tf.keras.layers.BatchNormalization()(x0)
    x0 = tf.keras.layers.Activation('relu')(x0)
    x0 = tf.keras.layers.Dropout(0.2)(x0)

    x1 = tf.keras.layers.Embedding(vocab_size, embedding_dim[1], input_length=max_len[1], mask_zero=True)(train_input_1)
    x1 = tf.keras.layers.Dropout(0.2)(x1)
    
    x1 = tf.keras.layers.Conv1D(embedding_dim[1], kernel_size=3, padding='valid', activation='relu')(x1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.Dropout(0.2)(x1)
    x1 = tf.keras.layers.GlobalMaxPooling1D()(x1)

    x1 = tf.keras.layers.Dense(128, activation=None)(x1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.Dropout(0.2)(x1)

    fc = tf.keras.layers.Concatenate()([x0,x1])

    fc = tf.keras.layers.Dense(128, activation=None)(fc)
    fc = tf.keras.layers.BatchNormalization()(fc)
    fc = tf.keras.layers.Activation('relu')(fc)
    fc = tf.keras.layers.Dropout(0.2)(fc)

    labels = tf.keras.layers.Dense(num_labels, activation = 'softmax')(fc)
    
    model = tf.keras.Model([train_input_0,train_input_1], labels)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer = optimizer, loss = focal_loss_with_out_logit, metrics = ['accuracy'])
    
    return model

In [ ]:
from PIL import Image

model = convolutional_model()
tf.keras.utils.plot_model(model, dpi = 500, show_shapes=True)
img = Image.open('model.png')
img

In [ ]:
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 15)
mc = ModelCheckpoint('best_model2.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

model2 = convolutional_model()


num_epochs = 200

history = model2.fit(model2_train_input, labels, epochs=num_epochs, batch_size=1024, verbose=1, validation_split=0.2, callbacks=[es, mc])

Epoch 1/200
195/195 [==============================] - 61s 302ms/step - loss: 0.7670 - accuracy: 0.7874 - val_loss: 4.3872 - val_accuracy: 0.0000e+00

Epoch 00001: val_accuracy improved from -inf to 0.00000, saving model to best_model2.h5
Epoch 2/200
195/195 [==============================] - 58s 298ms/step - loss: 0.1365 - accuracy: 0.9254 - val_loss: 3.3800 - val_accuracy: 0.0000e+00

Epoch 00002: val_accuracy did not improve from 0.00000
Epoch 3/200
195/195 [==============================] - 58s 298ms/step - loss: 0.0665 - accuracy: 0.9550 - val_loss: 1.0166 - val_accuracy: 0.2330

Epoch 00003: val_accuracy improved from 0.00000 to 0.23300, saving model to best_model2.h5
Epoch 4/200
195/195 [==============================] - 58s 298ms/step - loss: 0.0447 - accuracy: 0.9686 - val_loss: 0.0859 - val_accuracy: 0.9291

Epoch 00004: val_accuracy improved from 0.23300 to 0.92905, saving model to best_model2.h5
Epoch 5/200
195/195 [==============================] - 58s 298ms/step - loss: 0

In [ ]:
model2.load_weights('best_model2.h5')

model2_pred=model2.predict(model2_test_input)
model2_pred=tf.argmax(model2_pred, axis=1)
sample_submission['model2_pred']=model2_pred

In [ ]:
# 모델 3 

train_length = train['filter_요약문_연구목표'].apply(len)

print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))

number = 98
print('인풋 길이 {}%: {}'.format(number,np.percentile(train_length, number)))

인풋 길이 제 1 사분위: 24.0
인풋 길이 제 3 사분위: 72.0
인풋 길이 98%: 178.0


In [ ]:
def len_cut(text):
  return text[:85]

train['cut_요약문_연구목표'] = train['filter_요약문_연구목표'].apply(len_cut)
test['cut_요약문_연구목표'] = test['filter_요약문_연구목표'].apply(len_cut)

In [ ]:
train['model3_input'] = train['filter_내역사업명'] + train['filter_과제명'] + train['filter_요약문_한글키워드'] + train['cut_요약문_연구목표']
test['model3_input'] = test['filter_내역사업명'] + test['filter_과제명'] + test['filter_요약문_한글키워드'] + test['cut_요약문_연구목표']

In [ ]:
tokenizer=Tokenizer(oov_token = 'OOV')

tokenizer.fit_on_texts(list(train['model3_input']))
word_vocab = tokenizer.word_index

vocab_size = len(word_vocab)+2

In [ ]:
# 단어 사전 크기 확인 

threshold = 6
total_cnt = len(tokenizer.word_index) 
rare_cnt = 0 
total_freq = 0 
rare_freq = 0 

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

print('예상 단어집합 크기 :',total_cnt - rare_cnt)
words_num = total_cnt - rare_cnt

단어 집합(vocabulary)의 크기 : 208013
등장 빈도가 5번 이하인 희귀 단어의 수: 144465
단어 집합에서 희귀 단어의 비율: 69.44998629893324
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.5191817582534144
예상 단어집합 크기 : 63548


In [ ]:
# 단어 사전 크기 설정

tokenizer=Tokenizer(num_words=words_num,oov_token = 'OOV')

tokenizer.fit_on_texts(list(train['model1_input']))
word_vocab = tokenizer.word_index

vocab_size = len(word_vocab)+2

In [ ]:
# 0이 아닌 데이터 2번 복사하여 추가, 추가로 오답률이 높은 label 2번 복사하여 추가 
temp = train.copy()
temp1 = temp[temp['label'] != 0]
temp2 = temp[(temp['label'] == 11)|(temp['label'] == 12)|(temp['label'] == 13)|(temp['label'] == 17)|(temp['label'] == 20)|(temp['label'] == 23)|(temp['label'] == 25)|(temp['label'] == 26)|(temp['label'] == 37)|(temp['label'] == 38)]

temp = pd.concat([temp, temp1], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp1], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp2], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp2], axis = 0).reset_index(drop = True)

In [ ]:
train_length = train['model3_input'].apply(len)

print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))

number = 98
print('인풋 길이 {}%: {}'.format(number,np.percentile(train_length, number)))

인풋 길이 제 1 사분위: 43.0
인풋 길이 제 3 사분위: 94.0
인풋 길이 98%: 114.0


In [ ]:
# 벡터라이징

max_len = 114

model3_train_encoded = tokenizer.texts_to_sequences(list(temp['model3_input']))
model3_train_input = pad_sequences(model3_train_encoded, maxlen=max_len, padding='post')

model3_test_encoded = tokenizer.texts_to_sequences(list(test['model3_input']))
model3_test_input = pad_sequences(model3_test_encoded, maxlen=max_len, padding='post')

# 레이블 설정 
labels=np.array(temp['label'])

In [ ]:
embedding_dim = 512

model3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len, mask_zero=True),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv1D(embedding_dim, kernel_size=5, padding='valid', activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
 
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.GlobalMaxPooling1D(),

    tf.keras.layers.Dense(128, activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(46, activation='softmax')
])

# 모델 컴파일
model3.compile(loss=focal_loss_with_out_logit,         
              optimizer= tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


print(model3.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 114, 512)          50850816  
_________________________________________________________________
dropout_17 (Dropout)         (None, 114, 512)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 110, 512)          1311232   
_________________________________________________________________
batch_normalization_12 (Batc (None, 110, 512)          2048      
_________________________________________________________________
activation_12 (Activation)   (None, 110, 512)          0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 110, 512)          0         
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 512)              

In [ ]:
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 20)
mc = ModelCheckpoint('best_model3.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

num_epochs = 250
history = model3.fit(model3_train_input, labels, epochs=num_epochs, batch_size=1024, verbose=1, validation_split=0.2, callbacks=[es, mc])

Epoch 1/250
195/195 [==============================] - 32s 158ms/step - loss: 0.6710 - accuracy: 0.7977 - val_loss: 6.5710 - val_accuracy: 0.0000e+00

Epoch 00001: val_accuracy improved from -inf to 0.00000, saving model to best_model3.h5
Epoch 2/250
195/195 [==============================] - 30s 155ms/step - loss: 0.1047 - accuracy: 0.9366 - val_loss: 4.1066 - val_accuracy: 0.0000e+00

Epoch 00002: val_accuracy did not improve from 0.00000
Epoch 3/250
195/195 [==============================] - 30s 155ms/step - loss: 0.0533 - accuracy: 0.9640 - val_loss: 0.7249 - val_accuracy: 0.4125

Epoch 00003: val_accuracy improved from 0.00000 to 0.41250, saving model to best_model3.h5
Epoch 4/250
195/195 [==============================] - 30s 156ms/step - loss: 0.0377 - accuracy: 0.9742 - val_loss: 0.2649 - val_accuracy: 0.7306

Epoch 00004: val_accuracy improved from 0.41250 to 0.73055, saving model to best_model3.h5
Epoch 5/250
195/195 [==============================] - 30s 155ms/step - loss: 0

KeyboardInterrupt: ignored

In [ ]:
model3.load_weights('best_model3.h5')

model3_pred=model3.predict(model3_test_input)
model3_pred=tf.argmax(model3_pred, axis=1)
sample_submission['model3_pred']=model3_pred

In [ ]:
def to_list(text):
  return [text]

sample_submission['list_model1_pred'] = sample_submission['model1_pred'].apply(to_list)
sample_submission['list_model2_pred'] = sample_submission['model2_pred'].apply(to_list)
sample_submission['list_model3_pred'] = sample_submission['model3_pred'].apply(to_list)

sample_submission['ensemle'] = sample_submission['list_model1_pred'] + sample_submission['list_model2_pred'] + sample_submission['list_model3_pred']

In [ ]:
def vote(text):
  C = Counter(text)
  sorted_C = sorted(C.items(), key=lambda x: (-x[1], x[0]))
  return sorted_C[0][0]

sample_submission['ensemle_result'] = sample_submission['ensemle'].apply(vote)
sample_submission['label'] = sample_submission['ensemle_result']

In [ ]:
submission = sample_submission[['index','label']]

submission.to_csv('submission.csv', index=False)